In [1]:
import numpy as np
import pandas
import json
import urllib
import string
from bs4 import BeautifulSoup
import os
import string
from pprint import pprint
#import seaborn as sb                                      # JR: What does the Seaborn library do? Also, needed to pip install it above
#from itertools import product                             # JR: Don't know this library either
#from matplotlib.collections import LineCollection
#import statsmodels
#import statsmodels.formula.api as smf
#from scipy.stats import chisquare
#from IPython.display import Image
thisdir = '/home/idies/workspace/Storage/raddick/afl/'
os.chdir(thisdir)
print('Done')


Done


In [2]:
players_df = pandas.read_csv(thisdir+'all_player_names.csv', encoding='utf-8', low_memory=False, index_col='player_id')
print('backing up...')
players_df_bk = players_df
players_df.sample(3)
len(players_df)

backing up...


12728

# Get game stats for each player

In [6]:
debug = 0

print('retrieving from backup...')
players_df = players_df_bk
allplayers = []
startrow = 12000
endrow = 12728
cnt = startrow
all_players_list = []
for ix, thislink in players_df['link'][startrow:endrow].iteritems():  # ITERATE PLAYERS
    if (debug == 1):
        print('Row {0:.0f}: Player ID {1:}: {2:}'.format(cnt, ix, thislink))
    else:
        if (np.mod(cnt, 50) == 0):
            print('Row {0:.0f}: Player ID {1:}: {2:}'.format(cnt, ix, thislink))
    thispage = urllib.request.urlopen(thislink)
    soup = BeautifulSoup(thispage,'html.parser',from_encoding='utf-8')
    alltables = soup.findAll('table')
    this_player_stats_list = []
    for thistable in alltables:    # ITERATE SEASONS/TEAMS
        badtable = False
        thishead = thistable.find('thead')
        headrows = thistable.find('thead').findAll('tr')
        thisteam = headrows[0].find('th').text.split('-')[0].strip()
        try:
            thisyear = int(headrows[0].find('th').text.split('-')[1].strip())
        except IndexError:
            badtable = True
        if (badtable == False):
            if (debug >= 1):
                print('{0:} {1:.0f}...'.format(thisteam,thisyear))
            this_season_stats_list = []
            datacols = []
            dataheaders = headrows[1].findAll('th')
            for thiscol in dataheaders:
                datacols.append(thiscol.text)
            bodyrows = thistable.findAll('tr')
            for thisbodyrow in bodyrows[2:-1]:   # ITERATE GAMES
                this_game_stats_dict = {}
                this_game_stats_dict['team'] = thisteam
                game_data_text_all = thisbodyrow.findAll('td')
                for j in range(0, len(game_data_text_all)):
                    if (game_data_text_all[j].text == '\xa0'):
                        this_game_stats_dict[datacols[j]] = 0
                    elif (datacols[j] != 'Opponent' and datacols[j] != 'R' and datacols[j] != 'Rd'):
                        try:
                            this_game_stats_dict[datacols[j]] = int(game_data_text_all[j].text)
                        except ValueError:
                            this_game_stats_dict[datacols[j]] = int(game_data_text_all[j].text[0:-1])
                    else:
                        this_game_stats_dict[datacols[j]] = game_data_text_all[j].text
                this_game_stats_bigdict = {}
                this_game_stats_bigdict[this_game_stats_dict['Rd']] = this_game_stats_dict
                del this_game_stats_bigdict[this_game_stats_dict['Rd']]['Gm']
                del this_game_stats_bigdict[this_game_stats_dict['Rd']]['Rd']
                this_season_stats_list.append(this_game_stats_bigdict)
            this_season_stats_bigdict = {}
            this_season_stats_bigdict[thisyear] = this_season_stats_list
            this_player_stats_list.append(this_season_stats_bigdict)
    this_player_stats_bigdict = {}
    this_player_stats_bigdict[ix] = this_player_stats_list
    all_players_list.append(this_player_stats_bigdict)
    cnt = cnt + 1
         #   pprint(this_season_stats_bigdict)
        #this_player_stats_list.append(this_season_stats_bigdict)
        #pprint(this_player_stats_list)
    #this_player_stats_list.append(this_season_stats_bigdict)
    #pprint(this_player_stats_list)
    #print('\n\n')
#            this_game_stats_bigdict[thisyear] = this_game_stats_dict
#        this_player_stats_list.append(this_game_stats_bigdict)
#    this_player_stats_bigdict = {}
#    this_player_stats_bigdict[ix] = this_player_stats_list
print('Writing outfile...')
with open('player_game_stats_12k.json', 'w') as f:
    json.dump(all_players_list, f)
print('done')
print('Done!')
#pprint(all_players_list)


retrieving from backup...
Row 12000: Player ID watec002: https://afltables.com/afl/stats/players/C/Chris_Waterson1.html
Row 12050: Player ID waugf101: https://afltables.com/afl/stats/players/F/Fred_Waugh.html
Row 12100: Player ID weidw001: https://afltables.com/afl/stats/players/W/Wayne_Weidemann.html
Row 12150: Player ID westf102: https://afltables.com/afl/stats/players/F/Fred_West0.html
Row 12200: Player ID whitc103: https://afltables.com/afl/stats/players/C/Clarrie_White.html
Row 12250: Player ID whitt101: https://afltables.com/afl/stats/players/T/Ted_Whitfield.html
Row 12300: Player ID wilkt101: https://afltables.com/afl/stats/players/T/Terry_Wilkins.html
Row 12350: Player ID willi102: https://afltables.com/afl/stats/players/I/Ian_Williams1.html
Row 12400: Player ID willg108: https://afltables.com/afl/stats/players/G/Gary_Williamson.html
Row 12450: Player ID wilse102: https://afltables.com/afl/stats/players/E/Eric_Wilson.html
Row 12500: Player ID wirab001: https://afltables.com/afl

In [ ]:
for thisone in allplayers:
    for pid, pdata in thisone.items():
        print('Player {0:}'.format(pid))
        for i in range(0, len(pdata)):
            for sid, sdata in pdata[i].items():
                print('Season {0:}'.format(sid))
                for j in range(0, len(sdata)):
                    for gid, gdata in sdata[j].items():
                        print('Game {0:}'.format(gid))
                        pprint(gdata)
                        print('\n')
            

In [ ]:
datacols = ['game_number', 'opponent', 'round', 'result', 'jersey_number', 'kicks', 'marks', 'handballs']
datacols += ['disposals', 'goals', 'behinds', 'hitouts', 'tackles', 'rebound50s', 'inside50s']
datacols += ['clearances', 'clangers', 'free_kicks_for', 'free_kicks_against', 'brownlow_votes']
datacols += ['contested_possessions', 'uncontested_possessions', 'contested_marks']
datacols += ['one_percenters', 'bounces', 'goal_assists', 'pct_game_played']

df = pandas.DataFrame(data=None, columns=datacols)
for i in range(0,1):#len(all_player_links)):
    #print(all_player_links[i])
    thisplayername = all_player_links[i].split('/')[-1][0:-5].replace('_',' ')
    thisurl = all_player_links[i]
    thispage = urllib.request.urlopen(thisurl)
    soup = BeautifulSoup(thispage,'html.parser',from_encoding='utf-8')
    print('Parsing player {0:,.0f}: {1:}...'.format(i, thisplayername))
    alltables = soup.findAll('table')
    thisyearstats = []
    for j in range(7, len(alltables)):
        thisyeardata = alltables[j]
        thisgamestats = {}
        print('Parsing data from {0:}...'.format(alltables[j].findAll('tr')[0].text[-4:]))
        allrows = thisyeardata.findAll('tr')[2:-1]
        for thisrow in allrows:
            allcells = thisrow.findAll('td')
            for k in range(0, len(allcells)-1):
                thisgamestats[datacols[k]] = allcells[k].text
            thisyearstats.append(thisgamestats)
        print(thisyearstats)

In [ ]:

movies = []
colnames = ['Release Date','Movie','Genre','Production Budget','Domestic Box Office to Date','Trailer']
nColumns = len(colnames)
starting_letters = string.printable[0:36]+string.printable[69]
#starting_letters = ['a']
for thisletter in starting_letters:
    thisurl = 'http://www.the-numbers.com/movies/letter/1'#+thisletter
    print(thisurl)
    thispage = urllib.request.urlopen(thisurl)
    soup = BeautifulSoup(thispage,'html.parser',from_encoding='utf-8')
    firstTable = soup.find('table')
    theseRows = soup.findAll('tr')
    for row in theseRows:
        thisrow = []
        cells = row.findAll('td')
        for cell in cells:
            if (cell.get_text() == u'\xa0'):
                thisrow.append(''.encode('utf-8'))
            else:
                thisrow.append(cell.get_text().replace(u'\xa0',' ').encode('utf-8'))
        if (len(thisrow) == 6):
            movies.append(thisrow)
movies_decoded = [[s.decode('utf-8') for s in t] for t in movies]
print('Done!')

In [ ]:
thenumbers = pd.DataFrame.from_dict(movies_decoded)
thenumbers.columns = ['releasedate','title','genre','budget','revenue','trailer']

thenumbers['releasedate'] = pd.to_datetime(thenumbers['releasedate'],format=u"%b\xa0%d,\xa0%Y")#,coerce=True)

thenumbers = thenumbers.replace([u'',u'nan'],[np.nan,np.nan])

thenumbers['budget'] = thenumbers['budget'].dropna().str.replace('$','').apply(lambda x:int(x.replace(',','')))
thenumbers['budget'] = thenumbers['budget'].dropna().apply(lambda x: '${:,.0f}'.format(x))

thenumbers['revenue'] = thenumbers['revenue'].dropna().str.replace('$','').apply(lambda x:int(x.replace(',','')))
thenumbers['revenue'] = thenumbers['revenue'].dropna().apply(lambda x: '${:,.0f}'.format(x))

thenumbers['year'] = thenumbers['releasedate'].apply(lambda x:x.year)

thenumbers['has_trailer'] = thenumbers['trailer'].apply(lambda x: True if (x == 'Play') else False)

thenumbers = thenumbers.drop('trailer',axis=1)

thenumbers.index.name = 'id'

thenumbers.columns = ['releasedate_thenumbers','title_thenumbers','genre_thenumbers','budget_thenumbers','revenue_thenumbers','year_thenumbers','has_trailer_thenumbers']
thenumbers.head(30)

thenumbers.to_csv('revenue.csv',encoding='utf8')
print(thenumbers.shape)  # as of 12/6/2017: 30,547 rows x 7 columns
                         # as of 4/12/2018: 31,711 rows x 7 columns
print('Done')